<a href="https://colab.research.google.com/github/johanhoffman/methods-in-computational-science/blob/main/MICS_Direct_Methods_YinengWang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Yineng Wang**

# **Abstract**

This report includes the implementation and tests of sparse matrix-vector product, QR factorization and direct solver of systems of linear equations.

#**About the code**

This report is written by Yineng Wang based on a template by Johan Hoffman.

In [1]:
# Copyright (C) 2020,2021 Johan Hoffman (jhoffman@kth.se)
# Copyright (C) 2021 Yineng Wang (yineng@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# **Set up environment**

To have access to the neccessary modules you have to run this cell. The functions for inner product, matrix multiplication and Euclidean norm implemented in Lab 1 are used here as utility functions.

In [2]:
import unittest

import numpy as np

# **Introduction**

This laboratory assignment include the implementation of sparse matrix-vector product, QR factorization and direct solver of systems of linear equations.

QR factorization employs modified Gram-Schmidt iteration, and the solver is based on QR factorization.

The implementations are based on the lecture notes.

# **Method**

The implementations use NumPy arrays to represent objects in linear algebra (vectors and matrices).

## 1. Sparse Matrix-vector Product



Input: vector $x$, sparse (real, quadratic) matrix $A$: CRS arrays val, col_idx, row_ptr

Output: matrix-vector product $b=Ax$

The implementation is based on the lecture notes, section 5.8, as well as [the Wikipedia page](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_.28CSR.2C_CRS_or_Yale_format.29).
The input matrix is represented by a 3-tuple of `(val, col_idx, row_ptr)`.
Like Wikipedia's implementation, the elements in `val` are indexed from 0. Therefore, the first element in `row_ptr` is 0. This differs from the implementation in the lecture notes and is for avoiding index offsetting.

To implement the sparse matrix-vector product, we repeatedly compute the inner product of each row of $A$ and $x$, $(a_{i:}, x) = \sum_{1 \le j \le n, a_{ij} \neq 0} a_{ij} x_j$. The non-zero elements in $a_{i:}$ can be accessed via `val` and `col_idx`.

In [3]:
def sparse_mat_vec_prod(A, x):
    if len(A) != 3:
        raise ValueError('A must be of the form (val, col_idx, row_ptr).')
    val, col_idx, row_ptr = A
    nnz = row_ptr[-1]
    m = len(row_ptr) - 1

    b = [0] * m
    for row in range(m):
        for idx in range(row_ptr[row], row_ptr[row+1]):
            b[row] += val[idx] * x[col_idx[idx]]
    return b

## 2. QR Factorization

Input: (real quadratic) matrix $A$

Output: orthogonal matrix $Q$, upper triangular matrix $R$, such that $A=QR$

The implementation uses modified Gram-Schmidt iteration, and is based on Algoritem 5.3 in the lecture notes.

For each iteration, we construct a base vector $v_{:j} = a_{:j} - \sum_{i=1}^{j-1} (a_{:j}, q_{:i}) q_{:i}$, and by equation (5.6), (5.7) and (5.8) in the lecture notes, $v_{:j} = (\prod_{i=1}^{j-1} (I - q_{:i} q_{:i}^T)) a_{:j}$.  Each column of $Q$ can be computed as the normalization of $v_{:j}$, namely, $q_{:j} = v_{:j} / \Vert v_{:j} \Vert$. Each column vector $r_{:j}$ of $R$ is the coordinate of $a_{:j}$ under the basis $\{q_j\}_{j=1}^{n}$, hence is computed as $r_{ij} = (a_{:j}, q_{:i})$.

In [4]:
def qr_factorization(A):
    n = len(A)
    Q = np.zeros((n,n))
    R = np.zeros((n,n))

    for j in range(n):
        v = np.copy(A[:,j])
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], v)    # ith coordinate of A[:,j] under q
            v -= R[i,j] * Q[:,i]
        R[j,j] = np.linalg.norm(v)
        Q[:,j] = v / R[j,j]
    return Q, R

## 3. Direct Solver Ax=b

Input: (real, quadratic) matrix $A$, vector $b$

Output: vector $x=A^{-1}b$

If we compute a QR factorization of $A$, we have that

$Ax = b \Leftrightarrow QRx = b \Leftrightarrow Rx = Q^{-1}b = Q^T b$.

Since $R$ is an upper triangular matrix, it is easy to solve the system of linear equations $Rx = Q^T b$. We can apply the backward substitution algorithm (Algorithm 5.2) in the lecture notes.

In [5]:
def backward_substitution(U, b):
    n = len(U)
    if len(b) != n:
        raise ValueError('The shape of U and b are unaligned.')

    x = np.copy(b)
    for row in range(n-1, -1, -1):
        for col in range(n-1, row, -1):
            x[row] -= U[row,col] * x[col]
        x[row] /= U[row,row]
    return x


def direct_solve(A, b):
    Q, R = qr_factorization(A)
    return backward_substitution(R, np.dot(Q.T, b))

# **Results**

The tests are carried out in Python's `unittest` framework. The NumPy's implmentations are used as a comparison to make sure the results are correct.

Since there might be round-off errors, NumPy's `assert_almost_equal` is used, which checks up to 7 decimals by default, instead of `assert_equal`.

## 1. Sparse Matrix-vector Product

In [6]:
class TestSparseMatVecProd(unittest.TestCase):
    def arg_validation(self):
        A = np.array([[1, 2], [0, 1]])
        x = np.array([3.0, 4.0])
        np.testing.assert_raises(ValueError, sparse_mat_vec_prod, A, x)

    def test_correctness(self):
        A = np.array([[0, 0, 1.4, 0, 3.1], [2.1, 0, 0, 0, 0], [-7.2, 0, 0, -10, 0]])
        A_sparse = (
            np.array([1.4, 3.1, 2.1, -7.2, -10]),
            np.array([2, 4, 0, 0, 3]),
            np.array([0, 2, 3, 5], dtype=int)
        )
        x = np.array([3.5, -4.23, 1.6, 0.85, 1.4])
        p1 = sparse_mat_vec_prod(A_sparse, x)
        p2 = np.dot(A, x)
        np.testing.assert_almost_equal(p1, p2)

## 2. QR Factorization

For this part, NumPy's implementation `np.linalg.qr` can be different from ours, in that the selection of $Q$ and $R$ is not unique. Therefore, we only test the properties of $Q$ and $R$, namely, $Q$ must be an orthonormal matrix, and $R$ must be an upper triangular matrix.

In [7]:
class TestQRFactorization(unittest.TestCase):
    def test_correctness(self):
        A = np.array([[1.5, 1.0, 8], [5, 2.0, 0.75], [1, 4.1, 0]])
        Q, R = qr_factorization(A)
        np.testing.assert_almost_equal(A, np.dot(Q, R))    # A = QR
        np.testing.assert_almost_equal(np.linalg.inv(Q), Q.T)    # Q^-1 = Q^T
        np.testing.assert_almost_equal(R, np.triu(R))    # R is upper triangle

## 3. Direct Solver Ax=b

In [8]:
class TestDirectSolve(unittest.TestCase):
    def arg_validation(self):
        A = np.array([[1, 2], [0, 1]])
        b = np.array([3.0])
        np.testing.assert_raises(ValueError, direct_solve, A, b)

    def test_correctness(self):
        A = np.array([[5, 2.0, -0.75], [1.5, 1, 8], [1, 4.1, 0]])
        b = np.array([3.0, -1.75, -2.63])
        x1 = direct_solve(A, b)
        x2 = np.linalg.solve(A, b)
        np.testing.assert_almost_equal(x1, x2)

## Conduct the test

In [9]:
if __name__ == '__main__':
    unittest.main(argv=['no_arg'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.048s

OK


# **Discussion**

All testing cases are passed. The results are expected, under the assumption that round-off errors less than or equal to 7 decimals are tolerated.

According to the lectures notes, the modified Gram-Schmidt algorithm is numerically more stable than the classical Gram-Schmidt algorithm. This might require furthur validation.